In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [ ]:
path = 'D:\\processing\\win\\rad_andre'

print(os.listdir(path)[0])

In [ ]:
df = pd.read_csv(os.path.join(path,os.listdir(path)[0]),sep=' ')

In [ ]:
# get the 'name' of every column in df besides the first one that holds the date
dfcnames = list(df.columns[1:])

# split every string column name into two, get the first part and turn it into a int and return
dfcnames = list(map(lambda x: int(x.split('.')[0]), dfcnames))

# test to check that both have the same size minus the date column in dfcnames
print(len(dfcnames),len(df.columns))

In [ ]:
# https://stackoverflow.com/questions/38101009/changing-multiple-column-names-but-not-all-of-them-pandas-python

# replace every column name in df with the new one
df.rename(columns=dict(zip(df.columns[1:], dfcnames)), inplace=True)

df.rename(columns={'nan':'Date'}, inplace=True)

df

In [ ]:
for n,b in enumerate(bnames):
    
    print(f'Integrating band {b}')
    
    blst = [i for i in range(int(begin[n]),int(end[n])+1)]
    print('Using bands:',blst)
    df[b] =  df[df.columns.intersection(blst)].mean(axis=1,skipna=True)

In [ ]:
df[df.columns.intersection(bnames)]

In [ ]:
# plt.plot(df[df.columns.intersection(bnames)].iloc[0])
for n,row in enumerate(df.iterrows()):
    plt.plot(df[df.columns.intersection(bnames)].iloc[n])

## Converting Excel date to py DateTime

In [ ]:
# https://stackoverflow.com/questions/31359150/convert-date-from-excel-in-number-format-to-date-format-python

excel_date = 42139.23213
dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + int(excel_date) - 2)
print(dt)
hour, minute, second = floatHourToTime(excel_date % 1)
dt = dt.replace(hour=hour, minute=minute, second=second)
print(dt)
del dt

In [ ]:
for n,i in enumerate(df['Date']):
    print(n,':',excel2datetime(i))

In [ ]:
plt.plot(df.iloc[1][1:])

## Processing and saving all the datasets

### Initial Setup:

In [2]:
# DECLARING SENTINEL-3 DEFAULT VARIABLES

centre = [400,412.5,442.5,490,510,560,620,665,673.75,681.25,708.75,753.75,761.25,764.375,767.5,778.75,865,885,900,940,1020]

begin = [392.5,407.5,437.5,485,505,555,615,660,670,677.5,703.75,750,760,762.5,766.25,771.25,855,880,895,930,1000]

end = [407.5,417.5,447.5,495,515,565,625,670,677.5,685,713.75,757.5,762.5,766.25,768.75,786.25,875,890,905,950,1040]

bandwidth = [15,10,10,10,10,10,10,10,7.5,7.5,10,7.5,2.5,3.75,2.5,15,20,10,10,20,40]

bnames = ['Oa01','Oa02','Oa03','Oa04','Oa05','Oa06','Oa07','Oa08','Oa09','Oa10','Oa11','Oa12','Oa13','Oa14','Oa15','Oa16','Oa17','Oa18','Oa19','Oa20','Oa21']

In [3]:
def fix_rad_cols(df):
    # get the 'name' of every column in df besides the first one that holds the date
    dfcnames = list(df.columns[1:])
    # split every string column name into two, get the first part and turn it into a int and return
    dfcnames = list(map(lambda x: int(x.split('.')[0]), dfcnames))
    # replace every column name in df with the new one
    df.rename(columns=dict(zip(df.columns[1:], dfcnames)), inplace=True)
#     # rename the first column
#     df.rename(columns={'nan':'Dates'}, inplace=True)
    return df

In [4]:
def floatHourToTime(fh):
    h, r = divmod(fh, 1)
    m, r = divmod(r*60, 1)
    return (
        int(h),
        int(m),
        int(r*60),
    )

In [5]:
def excel2datetime(excel_date,string=False):
    dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + int(excel_date) - 2)
    hour, minute, second = floatHourToTime(excel_date % 1)
    dt = dt.replace(hour=hour, minute=minute, second=second)
    if string:
        dt = str(dt)
    return dt

### Creating new files:

In [10]:
path = 'D:\\processing\\win\\rad_andre'

for file in os.listdir(path):
    print(f'Processing dataset: {file}\n')
    df = pd.read_csv(os.path.join(path,file),sep=' ')
    df = fix_rad_cols(df)
    
    for n,b in enumerate(bnames):
        
        # gcreate a list of column names with the wavelenghts corresponding to the S3 spectral bands.
        blst = [i for i in range(int(begin[n]),int(end[n])+1)]
        
        print(f'\tGenerating the MEAN value of band {b} using bands:\n\t{blst}\n\n')
        df[b] =  df[df.columns.intersection(blst)].mean(axis=1,skipna=True)
    
    # copy only the S3 integrated bands from the old df
    newdf = df[df.columns.intersection(bnames)].copy()
    
    # convert dates from excel format and save them in temporary pd.series
    stringdates = df.nan.apply(excel2datetime,string=True)
    
    # insert temp pd.series into the first column of the new df
    newdf.insert(0,'Date',stringdates)
    
    # Saving.
    newname = file.split('.')[0] + '_s3.csv'
    fullsavepath = os.path.join(path, newname)
    print(f'Saving file: {fullsavepath}')
    newdf.to_csv(fullsavepath,index=False)
    print(f'Done.\n\n')
    

Processing dataset: Manacapuru_Rrs_09_18.txt

	Generating the MEAN value of band Oa01 using bands:
	[392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407]


	Generating the MEAN value of band Oa02 using bands:
	[407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417]


	Generating the MEAN value of band Oa03 using bands:
	[437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447]


	Generating the MEAN value of band Oa04 using bands:
	[485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495]


	Generating the MEAN value of band Oa05 using bands:
	[505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515]


	Generating the MEAN value of band Oa06 using bands:
	[555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565]


	Generating the MEAN value of band Oa07 using bands:
	[615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625]


	Generating the MEAN value of band Oa08 using bands:
	[660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670]


	Generating the MEAN value of ban

	Generating the MEAN value of band Oa13 using bands:
	[760, 761, 762]


	Generating the MEAN value of band Oa14 using bands:
	[762, 763, 764, 765, 766]


	Generating the MEAN value of band Oa15 using bands:
	[766, 767, 768]


	Generating the MEAN value of band Oa16 using bands:
	[771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786]


	Generating the MEAN value of band Oa17 using bands:
	[855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875]


	Generating the MEAN value of band Oa18 using bands:
	[880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890]


	Generating the MEAN value of band Oa19 using bands:
	[895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905]


	Generating the MEAN value of band Oa20 using bands:
	[930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950]


	Generating the MEAN value of band Oa21 using bands:
	[1000, 1001, 1002, 1003, 1004, 1005